In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from typing import Tuple, List, Set, Iterable, Dict, Any, Optional, Union, Callable
from collections import Counter

sns.set_style("darkgrid")

# Data

In [2]:
df = pd.read_csv("data/credit_score.csv", low_memory=False)
df.columns = map(str.lower, df.columns)
df.head(2)

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,...,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.82262,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.94496,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good


---

<div class="alert alert-info">
  <h1><center>Введение в `scikit-learn`</center></h1></div>

**Scikit-learn** - это библиотека, которая предоставляет богатый набор инструментов для подготовки данных и  решения широкого спектра задач машинного обучения.  
  
Основными преимуществами Scikit-learn являются:
- Простота использования и понимания: библиотека имеет простой и понятный интерфейс, ставший де-факто стандартом в индустрии машинного обучения
- Широкие возможности для предобработки данных: библиотека содержит инструменты для работы с различными типами данных и выполнения предварительной обработки данных перед обучением модели
- Разнообразие алгоритмов: библиотека предоставляет широкий спектр алгоритмов машинного обучения, таких как SVM, деревья решений, KNN, KMeans и другие
- Гибкость: Scikit-learn позволяет пользователю настроить параметры алгоритмов, что позволяет улучшить результаты модели
- Оптимизированность: в большинстве случаев базовые настройки алгоритмов обеспечивают высокую стартовую эффективность
- Высокая производительность: Scikit-learn использует библиотеки NumPy и SciPy, что обеспечивает быстрое выполнение вычислений
- Мобильность: натренированные модели легко сохранять и загружать из файлов, что позволяет создавать модели в одной среде и разворачивать её в других окружениях

## Основные типы конструкций в `scikit-learn`:

Большинство конструкций в `scikit-learn` наследуют от одного или нескольких из следующих классов:

- **Estimator**: озачает, что данный класс "оценивает" данные, т.е. умеет извлечь новую информацию из предоставленных данных. Данный этап реализуется при помощи метода **fit**, который принимает на вход аргументы *X*, *y* (опционально) и дополнительные параметры при необходимости:

![numerical](images/estimator.png)

- **Transformer**: озачает, что данный класс преобразует входящие данные, и возвращает датасет того же или модифицированного размера (может измениться количество столбцов, изменение количества строк категорически не приветствуется).
  
Данный этап реализуется при помощи метода **transform**, который принимает на вход аргументы *X*, *y* (опционально) и дополнительные параметры при необходимости:

![numerical](images/transformer.png)

- **Predictor**: озачает, что данный класс способен "делать выводы" из предоставленных данных.
  
Данный этап реализуется при помощи метода **predict**, который принимает на вход аргумент *X* и дополнительные параметры при необходимости. Также возможно (для задач классификации) вернуть не "предсказания", а вероятности для каждого из возможных классов:

![numerical](images/predictor.png)

Есть ещё множество дополнительных конструкций, которые дополняют и формируют структуру данной библиотеки.  
  
Также важно отметить, что множество из реализованных моделей являются комбинацией этих конструкций.  
  
Так, например, модель дерева решений сперва должна "научится" и построить дерево из предоставленных данных при помощи метода **fit** (и, таким образом, она будет являться `Estimator`), а на этапе **predict** она будет применять полученные "знания" на новых данных (значит, эта модель также является `Predictor`).

---

<div class="alert alert-info">
  <h1><center>Feature Engineering</center></h1></div>

# <center>Nones</center>

![numerical](images/nones.png)

# Interpolation

In [8]:
df[["monthly_inhand_salary", "num_credit_inquiries"]].head(2)

,monthly_inhand_salary,num_credit_inquiries
0,1824.843333,4.0
1,NaN,4.0


#### Самостоятельная реализация

In [10]:
class InterpolationImputer:

    def __init__(self, method: 'str' = 'linear'):
        self.method = method

    def fit(self, X: pd.DataFrame, **params):
        return self

    def transform(self, X: pd.DataFrame, **params) -> pd.DataFrame:
        X = X.copy()
        X = pd.concat(
                        [
                            feature_values.interpolate(method=self.method, limit_area=None)
                            for feature_name, feature_values
                            in  X.items()
                        ],
                        axis=1
                    )
        return X


In [12]:
ii = InterpolationImputer()
ii.transform(df[["monthly_inhand_salary", "num_credit_inquiries"]]).head(2)

,monthly_inhand_salary,num_credit_inquiries
0,1824.843333,4.0
1,1824.843333,4.0


#### Тот же трансформер, но работающий отдельно по каждой группе `groupby`:

In [26]:
class GroupbyInterpolationImputer:

    def __init__(self, by: str, method: 'str' = 'linear'):
        self.by = by
        self.method = method

    def fit(self, X: pd.DataFrame, **params):
        return self

    def transform(self, X: pd.DataFrame, **params) -> pd.DataFrame:
        return pd.concat([
                            InterpolationImputer(method=self.method).transform(X=group_df, **params)
                            for group_name, group_df
                            in  X.groupby(self.by, group_keys=False)
                        ]).loc[X.index]


In [29]:
X = df[["customer_id", "monthly_inhand_salary", "num_credit_inquiries"]]

In [30]:
gii = GroupbyInterpolationImputer(by="customer_id")
gii.transform(X).head(2)

,customer_id,monthly_inhand_salary,num_credit_inquiries
0,CUS_0xd40,1824.843333,4.0
1,CUS_0xd40,1824.843333,4.0


---

# <center>Extreme values</center>

![numerical](images/extreme.png)

---

# <center>Numerical Features</center>

![numerical](images/distribution.png)

# Normalization (Min-Max scaler)

In [28]:
df[["credit_utilization_ratio", "monthly_inhand_salary"]].head(2)

,credit_utilization_ratio,monthly_inhand_salary
0,26.82262,1824.843333
1,31.94496,NaN


#### Самостоятельная реализация

In [30]:
class CustomMinMaxScaler:

    def __init__(self):
        self.mins: pd.Series
        self.maxs: pd.Series

    def fit(self, X: pd.DataFrame, y=None, **params):
        self.mins = X.min(axis=0)
        self.maxs = X.max(axis=0)
        return self

    def transform(self, X: pd.DataFrame, **params) -> pd.DataFrame:
        X = X.copy()
        X = (X - self.mins) / (self.maxs - self.mins)
        return X
        
    def fit_transform(self, X: pd.DataFrame, y=None, **params) -> pd.DataFrame:
        return self.fit(X, y, **params).transform(X, **params)

In [31]:
custom_mms = CustomMinMaxScaler()
custom_mms.fit(df[["credit_utilization_ratio", "monthly_inhand_salary"]].head(30))
custom_mms.transform(df[["credit_utilization_ratio", "monthly_inhand_salary"]]).head(2)

,credit_utilization_ratio,monthly_inhand_salary
0,0.223586,0.0
1,0.490862,NaN


#### Класс scikit-learn

In [32]:
from sklearn.preprocessing import MinMaxScaler

In [33]:
mms = MinMaxScaler(feature_range=(0, 1), clip=True).set_output(transform="pandas")

In [35]:
mms.fit_transform(df[["credit_utilization_ratio", "monthly_inhand_salary"]].head(30)).head(2)

,credit_utilization_ratio,monthly_inhand_salary
0,0.223586,0.0
1,0.490862,NaN


Атрибуты, выученные на этапе `fit`:

In [38]:
mms.data_range_, mms.data_min_, mms.data_max_

(array([   19.16498039, 10362.37666667]),
 array([  22.53759303, 1824.84333333]),
 array([   41.70257342, 12187.22      ]))

---

# <center>Categorical Features</center>

![numerical](images/encoding.png)

## One-Hot encoding

In [10]:
df[["occupation", "credit_score"]].head(2)

,occupation,credit_score
0,Scientist,Good
1,Scientist,Good


#### Самостоятельная реализация

In [17]:
class CustomOneHotEncoder:

    def __init__(self, dummy_na: bool = True):
        self.dummy_na = dummy_na
        self.columns_ : List[str]

    def fit(self, X: pd.DataFrame, y=None, **params):
        # Запоминаем, какие признаки будут получены на этапе трансформации:
        self.columns_ = pd.get_dummies(X).columns
        return self

    def transform(self, X: pd.DataFrame, **params) -> pd.DataFrame:
        X = X.copy()
        # Кодируем значения в виде бинарных признаков:
        transformed = pd.get_dummies(X, dummy_na=self.dummy_na)
        # Заполняем отсутствующие кодировки:
        missing_catergories = set(self.columns_).difference(transformed.columns)
        for missing_catergory in missing_catergories:
            transformed.insert(0, missing_catergory, 0)
        # Возвращаем только те признаки, которые были выучены на этапе `fit`
        return transformed.loc[:, self.columns_]


In [19]:
custom_ohe = CustomOneHotEncoder()
custom_ohe.fit(df[["occupation", "credit_score"]].head(30))
custom_ohe.transform(df[["occupation", "credit_score"]]).head(2)

,occupation_Engineer,occupation_Entrepreneur,occupation_Scientist,occupation_Teacher,occupation________,credit_score_Good,credit_score_Standard
0,0,0,1,0,0,1,0
1,0,0,1,0,0,1,0


#### Класс scikit-learn

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [23]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore").set_output(transform="pandas")

In [24]:
ohe.fit(df[["occupation", "credit_score"]].head(30)).transform(df[["occupation", "credit_score"]]).head(2)

,occupation_Engineer,occupation_Entrepreneur,occupation_Scientist,occupation_Teacher,occupation________,credit_score_Good,credit_score_Standard
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0


Атрибуты, выученные на этапе `fit`:

In [15]:
ohe.feature_names_in_

array(['occupation', 'credit_score'], dtype=object)

In [14]:
ohe.categories_

[array(['Accountant', 'Architect', 'Developer', 'Doctor', 'Engineer',
        'Entrepreneur', 'Journalist', 'Lawyer', 'Manager', 'Mechanic',
        'Media_Manager', 'Musician', 'Scientist', 'Teacher', 'Writer',
        '_______'], dtype=object),
 array(['Good', 'Poor', 'Standard'], dtype=object)]

---

## Multi-Label encoding

In [40]:
loans = df["type_of_loan"].str.replace(pat=r"\sLoan|\sand", repl="", case=False, regex=True).str.split(", ")
loans.head(2)

0    [Auto, Credit-Builder, Personal, Home Equity]
1    [Auto, Credit-Builder, Personal, Home Equity]
Name: type_of_loan, dtype: object

#### Самостоятельная реализация

In [47]:
class CustomMultiLabelEncoder:

    MISSING_CODE = "<missing>"

    def __init__(self, encode_missing: bool = True):
        self.encode_missing = encode_missing
        self.features_ : List[str]

    def fit(self, X: pd.Series):
        self.features_ = sorted(set.union(*X.dropna().apply(set).values))
        return self

    def transform(self, X: pd.Series) -> pd.DataFrame:
        count_dict = X.apply(lambda val: Counter(val) if isinstance(val, list) else Counter()).to_dict()
        count_df = pd.DataFrame.from_dict(count_dict, orient="index")
        # Выравниваем по столбцам и по строкам:
        target_df = count_df.align(pd.DataFrame(0, index=X.index, columns=self.features_), fill_value=0)[0]
        # Убираем лишние столбцы:
        target_df = target_df.loc[:, self.features_].fillna(0).astype(int)
        # Добавляем колонку для индикации отсутствующих значений:
        if self.encode_missing:
            target_df[self.MISSING_CODE] = X.isna().astype(int)
        # Добавляем префикс:
        target_df = target_df.add_prefix(f"{X.name}_")
        return target_df

In [48]:
custom_mle = CustomMultiLabelEncoder()
custom_mle.fit(loans)

In [49]:
custom_mle.features_

['Auto',
 'Credit-Builder',
 'Debt Consolidation',
 'Home Equity',
 'Mortgage',
 'Not Specified',
 'Payday',
 'Personal',
 'Student']

In [51]:
custom_mle.transform(loans).head(2)

,type_of_loan_Auto,type_of_loan_Credit-Builder,type_of_loan_Debt Consolidation,type_of_loan_Home Equity,type_of_loan_Mortgage,type_of_loan_Not Specified,type_of_loan_Payday,type_of_loan_Personal,type_of_loan_Student,type_of_loan_<missing>
0,1,1,0,1,0,0,0,1,0,0
1,1,1,0,1,0,0,0,1,0,0


#### Класс scikit-learn

In [52]:
from sklearn.preprocessing import MultiLabelBinarizer

`MultiLabelBinarizer` не умеет работать с отсутствующими значениями, поэтому для демонстрации мы их выкинем:

In [53]:
mle = MultiLabelBinarizer()
mle.fit(loans.dropna())

MultiLabelBinarizer()

In [54]:
mle.classes_

array(['Auto', 'Credit-Builder', 'Debt Consolidation', 'Home Equity',
       'Mortgage', 'Not Specified', 'Payday', 'Personal', 'Student'],
      dtype=object)

Также, `MultiLabelBinarizer` не имеет встроенной возможности возвращать **pd.DataFrame**, поэтому это придётся делать вручную:

In [56]:
pd.DataFrame(data=mle.transform(loans.dropna()), index=loans.dropna().index, columns=mle.classes_).head(2)

,Auto,Credit-Builder,Debt Consolidation,Home Equity,Mortgage,Not Specified,Payday,Personal,Student
0,1,1,0,1,0,0,0,1,0
1,1,1,0,1,0,0,0,1,0


#### Самостоятельная реализация трансформера, который умеет обрабатывать сразу несколько признаков:

In [57]:
class MultilabelColumnEncoder:

    MISSING_INDICATOR = "<missing>"
    
    def __init__(self, encode_missing: bool = True):
        self.encode_missing = encode_missing
        self.encoders_ : Dict[str, MultiLabelBinarizer]

    def fit(self, X: pd.DataFrame, **params):
        # Создаём пустой словарь, где будем хранить трансформеры для каждого признака:
        self.encoders_ = {}
        # Перебираем каждый признак, создаём и обучаем транформер для него:
        for feature_name, feature_values in X.items():
            feature_binarizer = MultiLabelBinarizer()
            feature_binarizer.fit(feature_values.dropna())
            self.encoders_[feature_name] = feature_binarizer
        return self

    def transform(self, X: pd.DataFrame, **params) -> pd.DataFrame:
        return pd.concat(
                            [
                                self._apply_binarizer(
                                                        feature_name   = feature_name,
                                                        feature_values = feature_values
                                                    )
                                for feature_name, feature_values
                                in  X.items()
                            ],
                            axis = 1
                        ).loc[X.index]

    def _apply_binarizer(self, feature_name: str, feature_values: pd.Series) -> pd.DataFrame:
        # Извлекаем трансформер для заданного признака:
        binarizer = self.encoders_[feature_name]
        # Временно удаляем пустые значения:
        y = feature_values.dropna()
        # Создаём массив с кодированными значениями:
        data = binarizer.transform(y)
        # Преобразовываем массив в датафрейм:
        df = pd.DataFrame(data=data, index=y.index, columns=binarizer.classes_)
        # Восстанавливаем пропущенные строки
        df = df.align(feature_values, axis="index", join="right")[0].fillna(0).astype(int)
        # Добавляем (если задано) индикатор пропущенных значений:
        if self.encode_missing:
            df[self.MISSING_INDICATOR] = feature_values.isna().astype(int)
        # Добавляем префикс и возвращаем результат:
        df = df.add_prefix(f"{feature_name}_")
        return df.loc[feature_values.index]


In [63]:
mce = MultilabelColumnEncoder()
mce.fit(loans.to_frame())
mce.transform(loans.to_frame()).head(2)

,type_of_loan_Auto,type_of_loan_Credit-Builder,type_of_loan_Debt Consolidation,type_of_loan_Home Equity,type_of_loan_Mortgage,type_of_loan_Not Specified,type_of_loan_Payday,type_of_loan_Personal,type_of_loan_Student,type_of_loan_<missing>
0,1,1,0,1,0,0,0,1,0,0
1,1,1,0,1,0,0,0,1,0,0
